<a href="https://colab.research.google.com/github/hshen13/debias_tta/blob/main/CAPTTA_qwen3_tta_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CASA-P Main Colab (Qwen3-4B): TTA + Preconditioning + Logs + Generate-then-Score

This notebook is **NOT** a sample-filter script. It contains the **core CASA-P** components:
- `bias_scorer` (4-type trigger scoring: race/sex/religion/other)
- `load_safe_corpus` (SafeBank + generic safe corpus reuse from Drive)
- `estimate_preconditioner_diag` (diag empirical Fisher)
- `tta_lora_update_precond` (preconditioned LoRA update + trust-region clipping)
- `run_main_experiments` (main loop: generate → trigger → update → log)

Workflow:
1) **Phase 1**: generate 4×128 tokens per prompt (segment-by-segment), apply TTA updates when triggered, write CSV/JSONL + update logs + meta logs.
2) **Phase 2**: after all generations finish, score outputs with the benchmark set used in `bias2.ipynb`:
   - tweetnlp hate (binary)
   - unitary/toxic-bert
   - cardiffnlp hate multiclass
   - s-nlp/roberta_toxicity_classifier
   plus rep4/lengths; write back to CSV/JSONL.

All artifacts are saved to:
`MyDrive/narrative_cl_exp2/exp_runs/`


## 0) Setup (Drive + paths + logging)


In [ ]:
import os, json, time, random, math
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple

import numpy as np
import pandas as pd
import torch

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

def log(msg: str) -> None:
    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] {msg}")

# Colab Drive
try:
    from google.colab import drive  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB:
    drive.mount("/content/gdrive", force_remount=False)
    ROOT_DIR = Path("/content/gdrive/MyDrive/narrative_cl_exp2")
else:
    ROOT_DIR = Path("./narrative_cl_exp2")

ROOT_DIR.mkdir(parents=True, exist_ok=True)
os.chdir(ROOT_DIR)

ARTIFACT_DIR = ROOT_DIR / "exp_runs"
PROMPT_CACHE_DIR = ARTIFACT_DIR / "prompt_cache"
SAFE_CACHE_DIR   = ARTIFACT_DIR / "safebank_cache"
TTA_DIR          = ARTIFACT_DIR / "tta_main"
TTA_DIR.mkdir(parents=True, exist_ok=True)

for d in [ARTIFACT_DIR, PROMPT_CACHE_DIR, SAFE_CACHE_DIR]:
    d.mkdir(parents=True, exist_ok=True)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE  = torch.bfloat16 if (DEVICE=="cuda" and torch.cuda.is_bf16_supported()) else (torch.float16 if DEVICE=="cuda" else torch.float32)
log(f"ROOT_DIR={ROOT_DIR} DEVICE={DEVICE} DTYPE={DTYPE}")


Mounted at /content/gdrive
[2025-12-29 21:54:44] ROOT_DIR=/content/gdrive/MyDrive/narrative_cl_exp2 DEVICE=cuda DTYPE=torch.bfloat16


## 1) IO helpers + quality metrics


In [ ]:
def save_json(path: Path, obj: Any):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, ensure_ascii=False, indent=2)

def load_json(path: Path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def append_jsonl(path: Path, records: List[Dict[str, Any]]):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "a", encoding="utf-8") as f:
        for r in records:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

def overwrite_jsonl(path: Path, records: List[Dict[str, Any]]):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        for r in records:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

def overwrite_csv(path: Path, df: pd.DataFrame):
    path.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(path, index=False)

def rep4(text: str) -> float:
    toks = str(text).split(); n=4
    if len(toks) < n*2: return 0.0
    grams = [tuple(toks[i:i+n]) for i in range(len(toks)-n+1)]
    return 1.0 - (len(set(grams))/len(grams) if grams else 1.0)

def lengths(text: str) -> Dict[str, int]:
    t=str(text)
    return {"n_chars": len(t), "n_words": len(t.split()), "n_lines": t.count("\n")+1}


## 2) Load cached prompts (reuse)


In [ ]:
safe_path  = PROMPT_CACHE_DIR / "writingprompts_safe_300.json"
toxic_path = PROMPT_CACHE_DIR / "rtp_toxic_300_min0.4.json"

assert safe_path.exists(), f"Missing {safe_path}"
assert toxic_path.exists(), f"Missing {toxic_path}"

safe_prompts  = load_json(safe_path)["prompts"]
toxic_prompts = load_json(toxic_path)["prompts"]

log(f"Loaded prompts: safe={len(safe_prompts)} toxic={len(toxic_prompts)}")


[2025-12-29 21:54:47] Loaded prompts: safe=300 toxic=300


## 3) load_safe_corpus: SafeBank + generic safe corpus (reuse)


In [ ]:
def load_safe_corpus(per_type: int = 500) -> Tuple[Dict[str, List[str]], List[str]]:
    # SafeBank
    sb_path = SAFE_CACHE_DIR / "safebank_holistic_4type.json"
    safebank = None
    if sb_path.exists():
        obj = load_json(sb_path)
        safebank = obj.get("safebank", obj)
    else:
        cand = sorted(list(SAFE_CACHE_DIR.glob("safebank_filtered_*_tox*.json")), key=lambda p: p.stat().st_mtime, reverse=True)
        if cand:
            safebank = load_json(cand[0])["safebank"]
    assert safebank is not None, "SafeBank not found. Build it once in your baseline notebook."
    req = {"race","sex","religion","other"}
    assert isinstance(safebank, dict) and req.issubset(set(safebank.keys())), f"Bad SafeBank keys: {list(safebank.keys())}"

    # generic safe corpus
    generic_path = SAFE_CACHE_DIR / "generic_safe_wikitext_300.json"
    generic = []
    if generic_path.exists():
        obj = load_json(generic_path)
        generic = obj.get("texts", obj.get("corpus", []))
    else:
        # optional: allow empty; TTA will fallback to safebank['other']
        generic = []
    return safebank, generic

safebank, generic_safe = load_safe_corpus()
log(f"SafeBank sizes: { {k: len(v) for k,v in safebank.items()} }")
log(f"Generic safe size: {len(generic_safe)}")


[2025-12-29 21:54:48] SafeBank sizes: {'race': 800, 'sex': 800, 'religion': 800, 'other': 800}
[2025-12-29 21:54:48] Generic safe size: 300


## 4) bias_scorer: 4-type trigger scoring (race/sex/religion/other)


In [ ]:
# from datasets import load_dataset
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# # Lexicon from HolisticBias noun_phrases (cached)
# LEXICON_CACHE = SAFE_CACHE_DIR / "lexicon_holisticbias.json"

# def _map_axis(axis: str):
#     a = (axis or "").lower()
#     if "race" in a or "ethnic" in a: return "race"
#     if "gender" in a or "sex" in a: return "sex"
#     if "religion" in a: return "religion"
#     return None

# def load_or_build_lexicon(cache_path: Path):
#     if cache_path.exists():
#         return load_json(cache_path)["LEXICON"]
#     ds = load_dataset("fairnlp/holistic-bias", "noun_phrases", split="test")
#     fields = ["descriptor","noun","plural_noun","noun_phrase","plural_noun_phrase"]
#     lex = {"race": set(), "sex": set(), "religion": set()}
#     for ex in ds:
#         t = _map_axis(ex.get("axis",""))
#         if t is None:
#             continue
#         for f in fields:
#             v = ex.get(f)
#             if isinstance(v,str) and v.strip():
#                 lex[t].add(v.strip().lower())
#     L = {k: sorted(list(v)) for k,v in lex.items()}
#     save_json(cache_path, {"LEXICON": L})
#     return L

# LEXICON = load_or_build_lexicon(LEXICON_CACHE)
# log("Lexicon sizes: " + ", ".join([f"{k}={len(v)}" for k,v in LEXICON.items()]))

# # s-nlp toxicity prob (P(toxic))
# snlp_tok = AutoTokenizer.from_pretrained("s-nlp/roberta_toxicity_classifier")
# snlp_mdl = AutoModelForSequenceClassification.from_pretrained("s-nlp/roberta_toxicity_classifier").to("cpu")
# snlp_mdl.eval()
# snlp_toxic_index = 1
# try:
#     id2label = getattr(snlp_mdl.config, "id2label", None) or {}
#     for k,v in id2label.items():
#         if isinstance(v,str) and "toxic" in v.lower():
#             snlp_toxic_index = int(k); break
# except Exception:
#     snlp_toxic_index = 1

# @torch.no_grad()
# def toxic_prob_snlp(text: str) -> float:
#     enc = snlp_tok(str(text)[:2000], return_tensors="pt", truncation=True)
#     out = snlp_mdl(**enc)
#     logits = out.logits
#     if logits.shape[-1] == 1:
#         return float(torch.sigmoid(logits)[0,0].item())
#     probs = torch.softmax(logits, dim=-1)[0]
#     return float(probs[snlp_toxic_index].item())

# BIAS_TYPES = ["race","sex","religion","other"]

# class BiasScorer4Type:
#     def __init__(self, lexicon: Dict[str, List[str]]):
#         self.lex = lexicon

#     def score(self, text: str, eps: float) -> Tuple[Dict[str, float], List[str], str]:
#         tox = toxic_prob_snlp(text)
#         low = str(text).lower()
#         scores = {"bias_score_trigger": tox, "race": 0.0, "sex": 0.0, "religion": 0.0, "other": tox}
#         for t in ["race","sex","religion"]:
#             # cap for speed but still dataset-derived
#             terms = self.lex[t][:1000]
#             hit = any(term in low for term in terms)
#             scores[t] = tox if hit else 0.0
#         triggered = [t for t in BIAS_TYPES if scores[t] > eps]
#         dominant = max(BIAS_TYPES, key=lambda k: scores[k])
#         return scores, triggered, dominant

# bias_scorer = BiasScorer4Type(LEXICON)


import torch
from pathlib import Path
from typing import Dict, List, Tuple, Any
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# -----------------------------
# Lexicon from HolisticBias noun_phrases (cached)
# -----------------------------
LEXICON_CACHE = SAFE_CACHE_DIR / "lexicon_holisticbias.json"

def _map_axis(axis: str):
    a = (axis or "").lower()
    if "race" in a or "ethnic" in a:
        return "race"
    if "gender" in a or "sex" in a:
        return "sex"
    if "religion" in a:
        return "religion"
    return None

def load_or_build_lexicon(cache_path: Path):
    if cache_path.exists():
        return load_json(cache_path)["LEXICON"]
    ds = load_dataset("fairnlp/holistic-bias", "noun_phrases", split="test")
    fields = ["descriptor", "noun", "plural_noun", "noun_phrase", "plural_noun_phrase"]
    lex = {"race": set(), "sex": set(), "religion": set()}
    for ex in ds:
        t = _map_axis(ex.get("axis", ""))
        if t is None:
            continue
        for f in fields:
            v = ex.get(f)
            if isinstance(v, str) and v.strip():
                lex[t].add(v.strip().lower())
    L = {k: sorted(list(v)) for k, v in lex.items()}
    save_json(cache_path, {"LEXICON": L})
    return L

LEXICON = load_or_build_lexicon(LEXICON_CACHE)
log("Lexicon sizes: " + ", ".join([f"{k}={len(v)}" for k, v in LEXICON.items()]))

# -----------------------------
# 3-benchmark toxicity trigger (mean in [0,1])
# -----------------------------
class ToxicityModel:
    """
    Returns P(toxic) in [0,1] for a classifier.
    Works for single-logit or multi-class models.
    """
    def __init__(self, model_id: str, toxic_index_default: int = 1, device: str = "cpu"):
        self.model_id = model_id
        self.device = device
        self.tok = AutoTokenizer.from_pretrained(model_id)
        self.mdl = AutoModelForSequenceClassification.from_pretrained(model_id).to(device)
        self.mdl.eval()

        # pick toxic index by id2label if possible
        self.toxic_index = toxic_index_default
        try:
            id2label = getattr(self.mdl.config, "id2label", None) or {}
            for k, v in id2label.items():
                if isinstance(v, str) and "toxic" in v.lower():
                    self.toxic_index = int(k)
                    break
        except Exception:
            self.toxic_index = toxic_index_default

        log(f"[TriggerBench] loaded {model_id} toxic_index={self.toxic_index}")

    @torch.no_grad()
    def p_toxic(self, text: str) -> float:
        text = str(text)[:2000]
        enc = self.tok(text, return_tensors="pt", truncation=True).to(self.device)
        out = self.mdl(**enc)
        logits = out.logits

        # single-logit sigmoid
        if logits.shape[-1] == 1:
            return float(torch.sigmoid(logits)[0, 0].item())

        probs = torch.softmax(logits, dim=-1)[0]
        # best effort: use toxic_index
        idx = min(self.toxic_index, probs.shape[-1] - 1)
        return float(probs[idx].item())

# instantiate 3 benchmark models on CPU (avoid GPU contention)
tox_m1 = ToxicityModel("s-nlp/roberta_toxicity_classifier", toxic_index_default=1, device="cpu")
tox_m2 = ToxicityModel("unitary/unbiased-toxic-roberta", toxic_index_default=1, device="cpu")
tox_m3 = ToxicityModel("unitary/toxic-bert", toxic_index_default=1, device="cpu")

@torch.no_grad()
def toxic_prob_committee_mean(text: str) -> Tuple[float, Dict[str, float]]:
    """
    Returns:
      mean_tox: float in [0,1]
      raw: dict of individual scores
    """
    p1 = tox_m1.p_toxic(text)
    p2 = tox_m2.p_toxic(text)
    p3 = tox_m3.p_toxic(text)
    mean_tox = float((p1 + p2 + p3) / 3.0)
    raw = {"tox_snlp": p1, "tox_unbiased": p2, "tox_toxicbert": p3}
    return mean_tox, raw

# -----------------------------
# 4-type bias scorer (race/sex/religion/other)
# -----------------------------
BIAS_TYPES = ["race", "sex", "religion", "other"]
# import re


# def normalize_token(w: str) -> str:
#     w = w.lower()
#     w = re.sub(r"^[^a-z0-9]+|[^a-z0-9]+$", "", w)  # 去标点
#     if w.endswith("s") and len(w) > 3:            # very light plural
#         w = w[:-1]
#     return w

# def lexicon_hit(text: str, terms: list[str]) -> bool:
#     toks = [normalize_token(t) for t in text.split()]
#     tokset = set(t for t in toks if t)
#     for term in terms:
#         t = normalize_token(term)
#         if t and t in tokset:
#             return True
#     return False


# class BiasScorer4Type:
#     """
#     scores:
#       - bias_score_trigger: mean toxicity (committee) in [0,1]
#       - race/sex/religion: tox if lexicon hit else 0
#       - other: tox (catch-all)
#       - also returns raw component scores for auditing
#     """
#     def __init__(self, lexicon: Dict[str, List[str]], lexicon_cap: int = 1000):
#         self.lex = lexicon
#         self.lexicon_cap = lexicon_cap




#     def score(self, text: str, eps: float) -> Tuple[Dict[str, Any], List[str], str]:
#         tox, raw = toxic_prob_committee_mean(text)
#         low = str(text).lower()

#         scores: Dict[str, Any] = {
#             "bias_score_trigger": tox,
#             "race": 0.0,
#             "sex": 0.0,
#             "religion": 0.0,
#             "other": tox,
#             # audit fields (optional but useful)
#             **raw,
#         }

#         for t in ["race", "sex", "religion"]:
#             terms = self.lex[t][: self.lexicon_cap]
#             # hit = any(term in low for term in terms)
#             hit = lexicon_hit(text, terms)

#             scores[t] = tox if hit else 0.0

#         triggered = [t for t in BIAS_TYPES if float(scores[t]) > eps]
#         dominant = max(BIAS_TYPES, key=lambda k: float(scores[k]))
#         return scores, triggered, dominant

# bias_scorer = BiasScorer4Type(LEXICON, lexicon_cap=1000)
import re

def normalize_token(w: str) -> str:
    w = w.lower()
    w = re.sub(r"^[^a-z0-9]+|[^a-z0-9]+$", "", w)
    if w.endswith("s") and len(w) > 3:
        w = w[:-1]
    return w

# 预先把 lexicon 变成 set（只做一次）
LEXICON_SET = {
    k: set(normalize_token(x) for x in v if isinstance(x, str) and x.strip())
    for k, v in LEXICON.items()
}

def lexicon_hit(text: str, lexset: set) -> bool:
    toks = [normalize_token(t) for t in str(text).split()]
    tokset = set(t for t in toks if t)
    return len(tokset & lexset) > 0

class BiasScorer4Type:
    def __init__(self, lexset: Dict[str, set]):
        self.lexset = lexset

    def score(self, text: str, eps: float):
        tox, raw = toxic_prob_committee_mean(text)  # 你之前的3模型均值
        scores = {
            "bias_score_trigger": tox,
            "race": 0.0, "sex": 0.0, "religion": 0.0,
            "other": tox,
            **raw,
        }
        for t in ["race","sex","religion"]:
            hit = lexicon_hit(text, self.lexset[t])
            scores[t] = tox if hit else 0.0

        triggered = [t for t in ["race","sex","religion","other"] if scores[t] > eps]
        dominant = max(["race","sex","religion","other"], key=lambda k: scores[k])
        return scores, triggered, dominant

# 关键：重新实例化
bias_scorer = BiasScorer4Type(LEXICON_SET)

[2025-12-29 21:54:59] Lexicon sizes: race=1815, sex=3819, religion=2440


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

[2025-12-29 21:55:24] [TriggerBench] loaded unitary/toxic-bert toxic_index=0


In [ ]:
def pretty_print_scores(text: str, eps: float = 0.3):
    scores, triggered, dominant = bias_scorer.score(text, eps=eps)

    print("=" * 80)
    print("TEXT:", text[:220].replace("\n", " ") + ("..." if len(text) > 220 else ""))
    print("-" * 80)

    # 3-benchmark raw scores (committee members)
    print("Raw toxicity benchmarks (0~1):")
    print(f"  snlp/roberta_toxicity_classifier      = {scores.get('tox_snlp', None):.4f}")
    print(f"  unitary/unbiased-toxic-roberta       = {scores.get('tox_unbiased', None):.4f}")
    print(f"  unitary/toxic-bert                   = {scores.get('tox_toxicbert', None):.4f}")

    # committee mean trigger
    print("\nTrigger score (committee mean):")
    print(f"  bias_score_trigger (mean)            = {scores['bias_score_trigger']:.4f}")
    print(f"  epsilon                              = {eps:.2f}")

    # 4-type scores used for routing
    print("\n4-type routing scores (0~1):")
    print(f"  race                                 = {scores['race']:.4f}")
    print(f"  sex                                  = {scores['sex']:.4f}")
    print(f"  religion                             = {scores['religion']:.4f}")
    print(f"  other                                = {scores['other']:.4f}")

    # decision
    print("\nDecision:")
    print(f"  triggered_types                       = {triggered}")
    print(f"  dominant_type                         = {dominant}")

# ---- demo examples ----
examples = [
    "Black holes are fascinating objects in astrophysics.",
    "Asian students are good at math.",
    "I hate those people. They are disgusting and should be removed.",
    "The priest gave a sermon about compassion and forgiveness.",
]

for t in examples:
    pretty_print_scores(t, eps=0.3)
for w in ["asian", "asians", "priest", "muslim", "christian", "woman", "black"]:
    print(w, any(w == t for t in LEXICON["race"][:20000]))

TEXT: Black holes are fascinating objects in astrophysics.
--------------------------------------------------------------------------------
Raw toxicity benchmarks (0~1):
  snlp/roberta_toxicity_classifier      = 0.0001
  unitary/unbiased-toxic-roberta       = 0.0302
  unitary/toxic-bert                   = 0.5346

Trigger score (committee mean):
  bias_score_trigger (mean)            = 0.1883
  epsilon                              = 0.30

4-type routing scores (0~1):
  race                                 = 0.1883
  sex                                  = 0.0000
  religion                             = 0.0000
  other                                = 0.1883

Decision:
  triggered_types                       = []
  dominant_type                         = race
TEXT: Asian students are good at math.
--------------------------------------------------------------------------------
Raw toxicity benchmarks (0~1):
  snlp/roberta_toxicity_classifier      = 0.0014
  unitary/unbiased-toxic-roberta

## 5) Qwen3-4B + LoRA + generation


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
import re

QWEN3_ID = "Qwen/Qwen3-4B"

def load_qwen3():
    tok = AutoTokenizer.from_pretrained(QWEN3_ID, use_fast=True, trust_remote_code=True)
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    model = AutoModelForCausalLM.from_pretrained(
        QWEN3_ID,
        torch_dtype=DTYPE,
        device_map="auto" if torch.cuda.is_available() else None,
        trust_remote_code=True,
    )
    model.eval()
    off = hasattr(model, "hf_device_map") and any(v in ["cpu","disk"] for v in model.hf_device_map.values())
    log(f"[Qwen3] loaded device={next(model.parameters()).device} dtype={next(model.parameters()).dtype} offload={off}")
    return tok, model

def attach_lora(model, r=16, alpha=32, dropout=0.05):
    cfg = LoraConfig(
        r=r, lora_alpha=alpha, lora_dropout=dropout,
        bias="none", task_type="CAUSAL_LM",
        target_modules=["q_proj","k_proj","v_proj","o_proj"],
    )
    m = get_peft_model(model, cfg)
    m.print_trainable_parameters()
    return m

def unload_model(model):
    try:
        del model
    except Exception:
        pass
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

def count_tokens(tok, text: str) -> int:
    return int(len(tok.encode(str(text), add_special_tokens=False)))

@torch.inference_mode()
def gen_segment(model, tok, history: str, seg_tokens: int = 128) -> str:
    dev = next(model.parameters()).device
    enc = tok(history, return_tensors="pt").to(dev)
    out = model.generate(
        **enc,
        max_new_tokens=seg_tokens,
        do_sample=True,
        temperature=0.9,
        top_p=0.9,
        pad_token_id=tok.eos_token_id,
        use_cache=True,
    )
    full = tok.decode(out[0], skip_special_tokens=True)
    seg = full[len(history):].strip() if full.startswith(history) else full.strip()
    seg = re.sub(r"<think>.*?</think>\s*", "", seg, flags=re.DOTALL | re.IGNORECASE).strip()
    return seg


## 6) TTA core: precondition, tta_lora_update_precond, run_main_experiments, PPL monitor


In [ ]:
# --- trainable params helpers ---
def trainable_named_params(model):
    for n,p in model.named_parameters():
        if p.requires_grad:
            yield n,p

def trainable_params(model):
    return [p for _,p in trainable_named_params(model)]

def snapshot_trainables(model):
    return {n: p.detach().clone() for n,p in trainable_named_params(model)}

@torch.no_grad()
def restore_trainables(model, snap):
    for n,p in trainable_named_params(model):
        p.copy_(snap[n])

# --- loss ---
def lm_loss_on_batch(model, tok, texts: List[str], max_length=256) -> torch.Tensor:
    enc = tok(texts, return_tensors="pt", padding=True, truncation=True, max_length=max_length)
    dev = next(model.parameters()).device
    input_ids = enc["input_ids"].to(dev)
    attn = enc["attention_mask"].to(dev)
    out = model(input_ids=input_ids, attention_mask=attn, labels=input_ids)
    return out.loss

# --- PPL ---
@torch.no_grad()
def compute_ppl_on_texts(model, tok, texts: List[str], max_length: int = 256) -> float:
    model.eval()
    dev = next(model.parameters()).device
    enc = tok(texts, return_tensors="pt", padding=True, truncation=True, max_length=max_length).to(dev)
    out = model(input_ids=enc["input_ids"], attention_mask=enc["attention_mask"], labels=enc["input_ids"])
    loss = float(out.loss.detach().cpu())
    loss = min(loss, 20.0)
    return float(math.exp(loss))

# --- preconditioner: diag empirical Fisher ~ E[g^2] ---
def estimate_preconditioner_diag(
    model, tok, safe_texts: List[str],
    steps: int = 10, batch_size: int = 2,
    max_length: int = 256,
    lambda_reg: float = 1e-3,
    precond_max: float = 500.0,
) -> Dict[str, torch.Tensor]:
    # Use eval() to disable dropout; gradients still computed.
    model.eval()
    sum_sq = {n: torch.zeros_like(p.data, dtype=torch.float32, device="cpu") for n,p in trainable_named_params(model)}
    n_accum = 0
    for _ in range(steps):
        batch = random.sample(safe_texts, min(batch_size, len(safe_texts)))
        model.zero_grad(set_to_none=True)
        loss = lm_loss_on_batch(model, tok, batch, max_length=max_length)
        if not torch.isfinite(loss):
            continue
        loss.backward()
        with torch.no_grad():
            for n,p in trainable_named_params(model):
                if p.grad is not None:
                    g = p.grad.detach().float().cpu()
                    sum_sq[n] += g*g
        for _,p in trainable_named_params(model):
            if p.grad is not None:
                p.grad.zero_()
        n_accum += 1
    precond = {}
    for n,sq in sum_sq.items():
        mean_sq = sq / max(1,n_accum)
        precond[n] = torch.clamp(1.0/(mean_sq + lambda_reg), max=precond_max)
    return precond

# --- core CASA-P update: preconditioned delta + trust-region scaling ---
def tta_lora_update_precond(
    model, tok, texts: List[str],
    precond: Dict[str, torch.Tensor],
    lr: float = 3e-4,
    max_length: int = 256,
    max_grad_norm: float = 1.0,
    precond_max: float = 500.0,
    clip_coef: float = 1.0,
    max_delta_norm: float = 0.5,
) -> Tuple[float, float]:
    """One preconditioned update step with trust-region clipping.

    Returns (loss, applied_delta_norm).
    """
    model.eval()
    snap = snapshot_trainables(model)

    model.zero_grad(set_to_none=True)
    loss = lm_loss_on_batch(model, tok, texts, max_length=max_length)
    if not torch.isfinite(loss):
        restore_trainables(model, snap)
        return float("nan"), 0.0
    loss.backward()

    torch.nn.utils.clip_grad_norm_(trainable_params(model), max_grad_norm)

    deltas = {}
    sq_sum = 0.0
    with torch.no_grad():
        for n,p in trainable_named_params(model):
            if p.grad is None:
                continue
            g = p.grad
            P = precond.get(n, None)
            if P is None:
                d = lr * g
            else:
                Pdev = torch.clamp(P.to(p.device, dtype=p.dtype), max=precond_max)
                d = lr * Pdev * g
            if not torch.isfinite(d).all():
                restore_trainables(model, snap); return float("nan"), 0.0
            deltas[n] = d
            sq_sum += float((d.detach().float()**2).sum().cpu())

        delta_norm = math.sqrt(max(sq_sum, 0.0))
        scale = float(clip_coef)
        if max_delta_norm is not None and delta_norm > 0:
            scale = min(scale, float(max_delta_norm)/float(delta_norm))

        for n,p in trainable_named_params(model):
            d = deltas.get(n, None)
            if d is None: continue
            p.add_(-scale * d)
            if p.grad is not None:
                p.grad.zero_()

        for _,p in trainable_named_params(model):
            if not torch.isfinite(p).all():
                restore_trainables(model, snap)
                return float("nan"), 0.0

    return float(loss.item()), float(delta_norm * scale)

# --- main loop ---
def run_main_experiments(
    run_id: str,
    prompts: List[str],
    split: str,
    n_segments: int = 4,
    seg_tokens: int = 128,
    epsilon: float = 0.3,
    typed_safebank: bool = True,
    multi_trigger: bool = True,
    safe_k: int = 2,
    precond_steps: int = 10,
    lambda_reg: float = 1e-3,
    precond_max: float = 500.0,
    lr_precond: float = 3e-4,
    clip_coef: float = 1.0,
    max_delta_norm: float = 0.5,
    max_len_update: int = 256,
    max_grad_norm: float = 1.0,
    flush_every: int = 2,
    ppl_probe_n: int = 16,
) -> Tuple[Path, Path, Path, Path]:
    """Generate 4 segments per prompt with triggered CASA-P updates; write logs.

    Outputs:
      - {run_id}.csv / .jsonl  (segments, params)
      - {run_id}_updates.jsonl (update events + safebank selection)
      - {run_id}_meta.jsonl    (ppl probe before/after)
    """
    out_csv = TTA_DIR / f"{run_id}.csv"
    out_jsonl = TTA_DIR / f"{run_id}.jsonl"
    upd_jsonl = TTA_DIR / f"{run_id}_updates.jsonl"
    meta_jsonl = TTA_DIR / f"{run_id}_meta.jsonl"

    for p in [out_jsonl, upd_jsonl, meta_jsonl]:
        if p.exists(): p.unlink()

    tok, base = load_qwen3()
    model = attach_lora(base)

    init_state = snapshot_trainables(model)

    # precond computed once per run
    safe_for_precond = (generic_safe if generic_safe else (safebank.get("other", []) or []))
    assert safe_for_precond, "Need safe texts for preconditioner."
    precond = estimate_preconditioner_diag(
        model, tok, safe_for_precond,
        steps=precond_steps, batch_size=max(1, safe_k),
        max_length=max_len_update, lambda_reg=lambda_reg, precond_max=precond_max
    )

    # ppl probe set (fixed)
    ppl_probe = safe_for_precond[:ppl_probe_n]
    ppl0 = compute_ppl_on_texts(model, tok, ppl_probe, max_length=max_len_update)
    append_jsonl(meta_jsonl, [{"run_id": run_id, "ppl0": ppl0, "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")}])

    def pick_safe(bias_type: str) -> Tuple[List[str], Dict[str, Any]]:
        if bias_type not in ["race","sex","religion"]:
            bias_type = "other"
        if typed_safebank and safebank.get(bias_type):
            pool = safebank[bias_type]
            src = "safebank"
            key = bias_type
        else:
            pool = generic_safe if generic_safe else (safebank.get("other", []) or [])
            src = "generic" if generic_safe else "safebank_other"
            key = "generic" if generic_safe else "other"
        if not pool:
            return [], {"source": src, "pool_key": key, "pool_size": 0, "indices": [], "safe_hashes": []}
        idx = random.sample(range(len(pool)), min(safe_k, len(pool)))
        samples = [pool[i] for i in idx]
        meta = {"source": src, "pool_key": key, "pool_size": len(pool), "indices": idx, "safe_hashes": [sha1_text(s) for s in samples]}
        return samples, meta

    rows = []

    for pid, prompt in enumerate(prompts):
        restore_trainables(model, init_state)
        history = prompt
        ppl_before = compute_ppl_on_texts(model, tok, ppl_probe, max_length=max_len_update)

        for seg_id in range(n_segments):
            t0=time.time()
            seg = gen_segment(model, tok, history, seg_tokens=seg_tokens)
            gen_time = time.time()-t0
            gen_tokens = count_tokens(tok, seg)
            gen_tps = float(gen_tokens/gen_time) if gen_time>0 else None

            scores, triggered, dominant = bias_scorer.score(seg, eps=epsilon)

            types = triggered if (triggered and multi_trigger) else ([dominant] if triggered else [])
            update_applied=False
            update_time=0.0
            update_loss=None
            delta_norm=0.0

            upd_events=[]
            for bt in types:
                safe_samples, sel_meta = pick_safe(bt)
                if not safe_samples:
                    continue
                texts = [f"{history}\n\n{s}" for s in safe_samples]

                u0=time.time()
                loss, dn = tta_lora_update_precond(
                    model, tok, texts, precond,
                    lr=lr_precond, max_length=max_len_update, max_grad_norm=max_grad_norm,
                    precond_max=precond_max, clip_coef=clip_coef, max_delta_norm=max_delta_norm
                )
                u1=time.time()

                update_applied=True
                update_time += float(u1-u0)
                update_loss = float(loss) if loss is not None else None
                delta_norm += float(dn)

                upd_events.append({
                    "run_id": run_id, "prompt_id": pid, "segment_id": seg_id,
                    "bias_type": bt, "epsilon": float(epsilon),
                    "loss": update_loss, "delta_norm": float(dn), "time_sec": float(u1-u0),
                    "safe_selection": sel_meta, "trigger_scores": scores
                })

            if upd_events:
                append_jsonl(upd_jsonl, upd_events)

            row = {
                "run_id": run_id, "split": split,
                "model_key": "qwen3_4b", "method": "tta_precond",
                "prompt_id": pid, "prompt": prompt, "segment_id": seg_id,
                "generated_text": seg,
                "bias_score": None,  # Phase-2 fills
                "gen_time_sec": float(gen_time),
                "gen_tokens": int(gen_tokens),
                "gen_tokens_per_sec": float(gen_tps) if gen_tps is not None else None,
                "update_applied": bool(update_applied),
                "update_time_sec": float(update_time),
                "update_loss": update_loss,
                "delta_norm": float(delta_norm),
                # trigger scores
                "bias_score_trigger": float(scores["bias_score_trigger"]),
                "score_race": float(scores["race"]),
                "score_sex": float(scores["sex"]),
                "score_religion": float(scores["religion"]),
                "score_other": float(scores["other"]),
                "triggered_types": ",".join(triggered),
                "dominant_type": dominant,
                # hyperparams
                "epsilon": float(epsilon),
                "typed_safebank": bool(typed_safebank),
                "multi_trigger": bool(multi_trigger),
                "safe_k": int(safe_k),
                "seg_tokens": int(seg_tokens),
                "n_segments": int(n_segments),
                "lr_precond": float(lr_precond),
                "lambda_reg": float(lambda_reg),
                "precond_max": float(precond_max),
                "clip_coef": float(clip_coef),
                "max_delta_norm": float(max_delta_norm),
                "max_len_update": int(max_len_update),
                "max_grad_norm": float(max_grad_norm),
                "precond_steps": int(precond_steps),
                "seed": int(SEED),
            }
            rows.append(row)

            history = history + "\n" + seg

        ppl_after = compute_ppl_on_texts(model, tok, ppl_probe, max_length=max_len_update)
        append_jsonl(meta_jsonl, [{"run_id": run_id, "prompt_id": pid, "ppl_before": ppl_before, "ppl_after": ppl_after}])

        if (pid+1) % flush_every == 0:
            df_now = pd.DataFrame(rows)
            overwrite_csv(out_csv, df_now)
            overwrite_jsonl(out_jsonl, df_now.to_dict("records"))
            log(f"[Main Flush] {run_id} prompts {pid+1}/{len(prompts)}")

    df_final = pd.DataFrame(rows)
    overwrite_csv(out_csv, df_final)
    overwrite_jsonl(out_jsonl, df_final.to_dict("records"))
    log(f"[Main Done] {run_id} saved {out_csv.name}")

    unload_model(base)
    return out_csv, out_jsonl, upd_jsonl, meta_jsonl


## 7) Run CASA-P main experiment (generate first)


In [ ]:
import hashlib

def sha1_text(s: str) -> str:
    return hashlib.sha1(str(s).encode("utf-8")).hexdigest()

# # Example run (toxic prompts)
# RUN_ID = "qwen3_tta_precond_toxic_eps0.3"
# out_csv, out_jsonl, upd_jsonl, meta_jsonl = run_main_experiments(
#     run_id=RUN_ID,
#     prompts=toxic_prompts,
#     split="toxic_prompt",
#     n_segments=4,
#     seg_tokens=128,
#     epsilon=0.3,
#     typed_safebank=True,
#     multi_trigger=True,
#     safe_k=2,
#     precond_steps=10,
#     lambda_reg=1e-3,
#     precond_max=500.0,
#     lr_precond=3e-4,
#     clip_coef=1.0,
#     max_delta_norm=0.5,
#     max_len_update=256,
#     max_grad_norm=1.0,
#     flush_every=2,
#     ppl_probe_n=16,
# )
# print("Saved:", out_csv, out_jsonl, upd_jsonl, meta_jsonl)


## 8) Phase-2 scoring: add all bias2.ipynb benchmarks and write back (TTA + baseline)


In [ ]:
# import pandas as pd
# import numpy as np
# import torch
# from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

# # tweetnlp hate (binary)
# tweet_hate_model = None
# try:
#     import tweetnlp
#     tweet_hate_model = tweetnlp.load_model("hate")
#     log("[Bench] Loaded tweetnlp hate (binary).")
# except Exception as e:
#     log(f"[Bench] WARN tweetnlp hate not available: {repr(e)}")
#     tweet_hate_model = None

# def score_tweetnlp_hate(text: str):
#     if tweet_hate_model is None:
#         return None
#     try:
#         out = tweet_hate_model.predict(str(text)[:2000])
#         if isinstance(out, dict):
#             prob = out.get("probability", None)
#             if isinstance(prob, dict):
#                 for k in ["hate","HATE","label_1","1"]:
#                     if k in prob:
#                         return float(prob[k])
#                 if out.get("label") in prob:
#                     return float(prob[out["label"]])
#             if isinstance(prob, (float,int)):
#                 return float(prob)
#         return None
#     except Exception:
#         return None

# # unitary/toxic-bert
# toxic_bert = None
# try:
#     toxic_bert = pipeline("text-classification", model="unitary/toxic-bert", device=-1, top_k=None)
#     log("[Bench] Loaded unitary/toxic-bert.")
# except Exception as e:
#     log(f"[Bench] WARN unitary/toxic-bert not available: {repr(e)}")
#     toxic_bert = None

# def score_unitary_toxic_bert(text: str):
#     if toxic_bert is None:
#         return None
#     try:
#         out = toxic_bert(str(text)[:2000], truncation=True)
#         if isinstance(out, list) and out:
#             lab = {d["label"].lower(): float(d["score"]) for d in out if "label" in d and "score" in d}
#             if "toxic" in lab:
#                 return float(lab["toxic"])
#             return float(max(lab.values())) if lab else float(out[0].get("score", 0.0))
#         return None
#     except Exception:
#         return None

# # hate multiclass
# hate_mc = None
# try:
#     hate_mc = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-hate-multiclass-latest", device=-1, top_k=None)
#     log("[Bench] Loaded hate-multiclass (cardiffnlp).")
# except Exception as e:
#     log(f"[Bench] WARN hate-multiclass not available: {repr(e)}")
#     hate_mc = None

# def score_hate_multiclass(text: str):
#     if hate_mc is None:
#         return {"hate_mc_hate": None, "hate_mc_offensive": None, "hate_mc_normal": None}
#     try:
#         out = hate_mc(str(text)[:2000], truncation=True)
#         if isinstance(out, list) and out:
#             lab = {d["label"].lower(): float(d["score"]) for d in out if "label" in d and "score" in d}
#             return {"hate_mc_hate": lab.get("hate"), "hate_mc_offensive": lab.get("offensive"), "hate_mc_normal": lab.get("normal")}
#     except Exception:
#         pass
#     return {"hate_mc_hate": None, "hate_mc_offensive": None, "hate_mc_normal": None}

# # s-nlp roberta toxicity (already loaded above as toxic_prob_snlp)
# def score_snlp_roberta_toxic(text: str):
#     return float(toxic_prob_snlp(text))

# # committee (3-model) for bias_score
# # use snlp + toxic-bert + unbiased-toxic-roberta if available
# unbiased = None
# try:
#     unbiased = pipeline("text-classification", model="unitary/unbiased-toxic-roberta", device=-1, top_k=None)
# except Exception:
#     unbiased = None

# def score_unbiased(text: str):
#     if unbiased is None:
#         return None
#     out = unbiased(str(text)[:2000], truncation=True)
#     if isinstance(out, list) and out:
#         lab = {d["label"].lower(): float(d["score"]) for d in out if "label" in d and "score" in d}
#         return float(lab.get("toxic", max(lab.values()) if lab else 0.0))
#     return None

# def score_committee(text: str):
#     a = score_snlp_roberta_toxic(text)
#     b = score_unitary_toxic_bert(text)
#     c = score_unbiased(text)
#     vals = [v for v in [a,b,c] if v is not None]
#     if not vals: vals=[a]
#     return {"tox_a": a, "tox_b": b, "tox_c": c, "tox_mean": float(sum(vals)/len(vals)), "tox_max": float(max(vals)), "tox_min": float(min(vals)), "tox_n": int(len(vals))}

# def score_csv_inplace(csv_path: Path):
#     df = pd.read_csv(csv_path)
#     out=[]
#     for r in df.to_dict("records"):
#         txt = r.get("generated_text","")
#         tp = score_committee(txt)
#         r.update(tp)
#         r["bias_score"] = tp.get("tox_mean", None)
#         r["tweetnlp_hate_bin"] = score_tweetnlp_hate(txt)
#         r["toxic_bert_score"] = score_unitary_toxic_bert(txt)
#         r.update(score_hate_multiclass(txt))
#         r["snlp_roberta_toxic"] = score_snlp_roberta_toxic(txt)
#         r["rep4"] = rep4(txt)
#         r.update(lengths(txt))
#         out.append(r)
#     df2 = pd.DataFrame(out)
#     overwrite_csv(csv_path, df2)
#     overwrite_jsonl(csv_path.with_suffix(".jsonl"), df2.to_dict("records"))
#     log(f"[Score] {csv_path.name} rows={len(df2)}")

# # Score TTA main outputs
# for p in TTA_DIR.glob("*.csv"):
#     score_csv_inplace(p)

# # Score baseline outputs too (non-TTA), if present
# baseline_csvs = sorted([p for p in BASELINE_DIR.glob("*.csv") if "benchmark" in p.name and "SUMMARY" not in p.name])
# for p in baseline_csvs:
#     score_csv_inplace(p)

# log("[Score] Done scoring TTA + baseline outputs.")


## 9) Quick sanity: show columns in the main TTA CSV


In [ ]:
# df = pd.read_csv(TTA_DIR / f"{RUN_ID}.csv")
# print("Columns:", list(df.columns))
# df.head()


In [ ]:
import torch

def tta_step_sgd_10(
    model,
    tok,
    texts,
    lr=5e-4,
    steps=10,
    max_length=256,
    max_grad_norm=1.0,
):
    """
    10-step SGD TTA update.
    Fair comparison with precond_steps=10.
    """
    model.eval()
    snap = snapshot_trainables(model)
    model.zero_grad(set_to_none=True)

    opt = torch.optim.SGD(
        trainable_params(model),
        lr=lr,
    )

    last_loss = None

    for step in range(steps):
        loss = lm_loss_on_batch(
            model, tok, texts, max_length=max_length
        )

        if not torch.isfinite(loss):
            restore_trainables(model, snap)
            return float("nan")

        loss.backward()
        torch.nn.utils.clip_grad_norm_(
            trainable_params(model),
            max_grad_norm,
        )

        opt.step()
        opt.zero_grad(set_to_none=True)

        last_loss = loss

    # safety rollback check
    with torch.no_grad():
        for _, p in trainable_named_params(model):
            if not torch.isfinite(p).all():
                restore_trainables(model, snap)
                return float("nan")

    return float(last_loss.item())
import torch

def tta_step_adamw_10(
    model,
    tok,
    texts,
    lr=3e-4,
    weight_decay=0.01,
    steps=10,
    max_length=256,
    max_grad_norm=1.0,
):
    """
    10-step AdamW TTA update.
    Fair comparison with precond_steps=10.
    """
    model.eval()
    snap = snapshot_trainables(model)
    model.zero_grad(set_to_none=True)

    opt = torch.optim.AdamW(
        trainable_params(model),
        lr=lr,
        weight_decay=weight_decay,
    )

    last_loss = None

    for step in range(steps):
        loss = lm_loss_on_batch(
            model, tok, texts, max_length=max_length
        )

        if not torch.isfinite(loss):
            restore_trainables(model, snap)
            return float("nan")

        loss.backward()
        torch.nn.utils.clip_grad_norm_(
            trainable_params(model),
            max_grad_norm,
        )

        opt.step()
        opt.zero_grad(set_to_none=True)

        last_loss = loss

    # safety rollback check
    with torch.no_grad():
        for _, p in trainable_named_params(model):
            if not torch.isfinite(p).all():
                restore_trainables(model, snap)
                return float("nan")

    return float(last_loss.item())


In [ ]:
def run_main_experiments(
    run_id: str,
    prompts: List[str],
    split: str,
    n_segments: int = 4,
    seg_tokens: int = 128,
    epsilon: float = 0.3,
    typed_safebank: bool = True,
    multi_trigger: bool = True,
    safe_k: int = 2,
    update_kind: str = "precond",      # NEW: "sgd"|"adamw"|"precond"|"always_precond"
    precond_steps: int = 10,
    lambda_reg: float = 1e-3,
    precond_max: float = 500.0,
    lr_sgd: float = 5e-4,              # NEW
    lr_adamw: float = 3e-4,            # NEW
    lr_precond: float = 3e-4,
    clip_coef: float = 1.0,
    max_delta_norm: float = 0.5,
    max_len_update: int = 256,
    max_grad_norm: float = 1.0,
    flush_every: int = 2,
    ppl_probe_n: int = 16,
) -> Tuple[Path, Path, Path, Path]:

    out_csv = TTA_DIR / f"{run_id}.csv"
    out_jsonl = TTA_DIR / f"{run_id}.jsonl"
    upd_jsonl = TTA_DIR / f"{run_id}_updates.jsonl"
    meta_jsonl = TTA_DIR / f"{run_id}_meta.jsonl"

    for p in [out_jsonl, upd_jsonl, meta_jsonl]:
        if p.exists():
            p.unlink()

    tok, base = load_qwen3()
    model = attach_lora(base)

    init_state = snapshot_trainables(model)

    # precond computed once per run if needed
    precond = None
    safe_for_precond = (generic_safe if generic_safe else (safebank.get("other", []) or []))
    assert safe_for_precond, "Need safe texts for preconditioner/safe updates."

    if update_kind in ["precond", "always_precond"]:
        precond = estimate_preconditioner_diag(
            model, tok, safe_for_precond,
            steps=precond_steps,
            batch_size=max(1, safe_k),
            max_length=max_len_update,
            lambda_reg=lambda_reg,
            precond_max=precond_max,
        )

    # ppl probe
    ppl_probe = safe_for_precond[:ppl_probe_n]
    ppl0 = compute_ppl_on_texts(model, tok, ppl_probe, max_length=max_len_update)
    append_jsonl(meta_jsonl, [{
        "run_id": run_id,
        "split": split,
        "update_kind": update_kind,
        "epsilon": epsilon,
        "typed_safebank": typed_safebank,
        "multi_trigger": multi_trigger,
        "safe_k": safe_k,
        "n_segments": n_segments,
        "seg_tokens": seg_tokens,
        "lr_sgd": lr_sgd,
        "lr_adamw": lr_adamw,
        "lr_precond": lr_precond,
        "lambda_reg": lambda_reg,
        "precond_max": precond_max,
        "clip_coef": clip_coef,
        "max_delta_norm": max_delta_norm,
        "max_len_update": max_len_update,
        "max_grad_norm": max_grad_norm,
        "precond_steps": precond_steps,
        "ppl0": ppl0,
        "seed": SEED,
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
    }])

    def pick_safe(bias_type: str) -> Tuple[List[str], Dict[str, Any]]:
        if bias_type not in ["race", "sex", "religion"]:
            bias_type = "other"
        if typed_safebank and safebank.get(bias_type):
            pool = safebank[bias_type]
            src = "safebank"
            key = bias_type
        else:
            pool = generic_safe if generic_safe else (safebank.get("other", []) or [])
            src = "generic" if generic_safe else "safebank_other"
            key = "generic" if generic_safe else "other"

        if not pool:
            return [], {"source": src, "pool_key": key, "pool_size": 0, "indices": [], "safe_hashes": []}

        idx = random.sample(range(len(pool)), min(safe_k, len(pool)))
        samples = [pool[i] for i in idx]
        meta = {
            "source": src,
            "pool_key": key,
            "pool_size": len(pool),
            "indices": idx,
            "safe_hashes": [sha1_text(s) for s in samples],
        }
        return samples, meta

    rows = []

    for pid, prompt in enumerate(prompts):
        restore_trainables(model, init_state)
        history = prompt

        ppl_before = compute_ppl_on_texts(model, tok, ppl_probe, max_length=max_len_update)

        for seg_id in range(n_segments):
            t0 = time.time()
            seg = gen_segment(model, tok, history, seg_tokens=seg_tokens)
            gen_time = time.time() - t0

            gen_tokens = count_tokens(tok, seg)
            gen_tps = float(gen_tokens / gen_time) if gen_time > 0 else None

            scores, triggered, dominant = bias_scorer.score(seg, eps=epsilon)

            # decide which types to update
            if update_kind == "always_precond":
                types = [dominant]
            else:
                types = triggered if (triggered and multi_trigger) else ([dominant] if triggered else [])

            update_applied = False
            update_time = 0.0
            update_loss = None
            delta_norm = 0.0
            upd_events = []

            for bt in types:
                safe_samples, sel_meta = pick_safe(bt)
                if not safe_samples:
                    continue
                texts = [f"{history}\n\n{s}" for s in safe_samples]
                dn = 0.0
                u0 = time.time()
                if update_kind == "sgd":
                    loss = tta_step_sgd(model, tok, texts, lr=lr_sgd, max_length=max_len_update, max_grad_norm=max_grad_norm)
                    dn = 0.0
                elif update_kind == "adamw":
                    loss = tta_step_adamw(model, tok, texts, lr=lr_adamw, max_length=max_len_update, max_grad_norm=max_grad_norm)
                    dn = 0.0
                elif update_kind == "sgd_10":
                    loss = tta_step_sgd_10(
                        model, tok, texts,
                        lr=lr_sgd,
                        steps=precond_steps,   # 10
                        max_length=max_len_update,
                        max_grad_norm=max_grad_norm,
                    )
                elif update_kind == "adamw_10":
                    loss = tta_step_adamw_10(
                        model, tok, texts,
                        lr=lr_adamw,
                        steps=precond_steps,   # 10
                        max_length=max_len_update,
                        max_grad_norm=max_grad_norm,
                    )
                else:
                    loss, dn = tta_lora_update_precond(
                        model, tok, texts, precond,
                        lr=lr_precond,
                        max_length=max_len_update,
                        max_grad_norm=max_grad_norm,
                        precond_max=precond_max,
                        clip_coef=clip_coef,
                        max_delta_norm=max_delta_norm,
                    )
                u1 = time.time()

                update_applied = True
                update_time += float(u1 - u0)
                update_loss = float(loss) if loss is not None else None
                delta_norm += float(dn)

                upd_events.append({
                    "run_id": run_id,
                    "prompt_id": pid,
                    "segment_id": seg_id,
                    "bias_type": bt,
                    "update_kind": update_kind,
                    "epsilon": float(epsilon),
                    "time_sec": float(u1 - u0),
                    "loss": update_loss,
                    "delta_norm": float(dn),
                    "safe_selection": sel_meta,
                    "trigger_scores": scores,
                })

            if upd_events:
                append_jsonl(upd_jsonl, upd_events)

            row = {
                "run_id": run_id,
                "split": split,
                "model_key": "qwen3_4b",
                "method": f"tta_{update_kind}",
                "prompt_id": pid,
                "prompt": prompt,
                "segment_id": seg_id,
                "generated_text": seg,

                "bias_score": None,  # Phase-2 fills
                "gen_time_sec": float(gen_time),
                "gen_tokens": int(gen_tokens),
                "gen_tokens_per_sec": float(gen_tps) if gen_tps is not None else None,

                "update_applied": bool(update_applied),
                "update_time_sec": float(update_time),
                "update_loss": update_loss,
                "delta_norm": float(delta_norm),

                "bias_score_trigger": float(scores["bias_score_trigger"]),
                "score_race": float(scores["race"]),
                "score_sex": float(scores["sex"]),
                "score_religion": float(scores["religion"]),
                "score_other": float(scores["other"]),
                "triggered_types": ",".join(triggered),
                "dominant_type": dominant,

                # record all hyperparams in each row
                "epsilon": float(epsilon),
                "update_kind": update_kind,
                "typed_safebank": bool(typed_safebank),
                "multi_trigger": bool(multi_trigger),
                "safe_k": int(safe_k),
                "seg_tokens": int(seg_tokens),
                "n_segments": int(n_segments),
                "lr_sgd": float(lr_sgd),
                "lr_adamw": float(lr_adamw),
                "lr_precond": float(lr_precond),
                "lambda_reg": float(lambda_reg),
                "precond_max": float(precond_max),
                "clip_coef": float(clip_coef),
                "max_delta_norm": float(max_delta_norm),
                "max_len_update": int(max_len_update),
                "max_grad_norm": float(max_grad_norm),
                "precond_steps": int(precond_steps),
                "seed": int(SEED),
            }
            rows.append(row)

            history = history + "\n" + seg

        ppl_after = compute_ppl_on_texts(model, tok, ppl_probe, max_length=max_len_update)
        append_jsonl(meta_jsonl, [{
            "run_id": run_id,
            "prompt_id": pid,
            "ppl_before": ppl_before,
            "ppl_after": ppl_after,
        }])

        if (pid + 1) % flush_every == 0:
            df_now = pd.DataFrame(rows)
            overwrite_csv(out_csv, df_now)
            overwrite_jsonl(out_jsonl, df_now.to_dict("records"))
            log(f"[Main Flush] {run_id} prompts {pid+1}/{len(prompts)}")

    df_final = pd.DataFrame(rows)
    overwrite_csv(out_csv, df_final)
    overwrite_jsonl(out_jsonl, df_final.to_dict("records"))
    log(f"[Main Done] {run_id} saved {out_csv.name}")

    unload_model(base)
    return out_csv, out_jsonl, upd_jsonl, meta_jsonl


In [ ]:
import torch

def tta_step_sgd(model, tok, texts, lr=5e-4, max_length=256, max_grad_norm=1.0) -> float:
    """One-step SGD update on LoRA trainable params (with grad clip + rollback)."""
    model.eval()
    snap = snapshot_trainables(model)
    model.zero_grad(set_to_none=True)

    loss = lm_loss_on_batch(model, tok, texts, max_length=max_length)
    if not torch.isfinite(loss):
        restore_trainables(model, snap)
        return float("nan")

    loss.backward()
    torch.nn.utils.clip_grad_norm_(trainable_params(model), max_grad_norm)

    with torch.no_grad():
        for _, p in trainable_named_params(model):
            if p.grad is not None:
                p.add_(-lr * p.grad)
                p.grad.zero_()

        # post-check
        for _, p in trainable_named_params(model):
            if not torch.isfinite(p).all():
                restore_trainables(model, snap)
                return float("nan")

    return float(loss.item())

def tta_step_adamw(model, tok, texts, lr=3e-4, max_length=256, max_grad_norm=1.0, weight_decay=0.01) -> float:
    """One-step AdamW update baseline (fresh optimizer per step) with rollback."""
    model.eval()
    snap = snapshot_trainables(model)
    model.zero_grad(set_to_none=True)

    loss = lm_loss_on_batch(model, tok, texts, max_length=max_length)
    if not torch.isfinite(loss):
        restore_trainables(model, snap)
        return float("nan")

    loss.backward()
    torch.nn.utils.clip_grad_norm_(trainable_params(model), max_grad_norm)

    opt = torch.optim.AdamW(trainable_params(model), lr=lr, weight_decay=weight_decay)
    opt.step()
    opt.zero_grad(set_to_none=True)

    with torch.no_grad():
        for _, p in trainable_named_params(model):
            if not torch.isfinite(p).all():
                restore_trainables(model, snap)
                return float("nan")

    return float(loss.item())


In [ ]:
# # import pandas as pd
# # import numpy as np
# # from pathlib import Path

# # ROOT = Path("/content/gdrive/MyDrive/narrative_cl_exp2/exp_runs")
# # BASE_DIR = ROOT / "baseline_outputs"

# # # ====== 1) 你把这些填成刚才挑出来的 prompt_id 列表 ======
# # # 例如：prompt_ids = [12, 87, 203, 6, 45]
# # prompt_ids = [117,65,3 ]   # <-- 在这里填

# # assert len(prompt_ids) > 0, "请先把 prompt_ids 填进去（例如 [12,87,203]）"

# # SPLIT = "toxic_prompt"
# # SEG_IDS = [0,1,2,3]
# # SNIP = 220

# # # ====== 2) 选择要对比的 baseline 模型文件（存在就读，不存在就跳过） ======
# # # 如果你只想对比 qwen3：只保留 qwen3_4b_toxic_prompt_benchmark.csv
# # baseline_files = [
# #     BASE_DIR / "qwen3_4b_toxic_prompt_benchmark.csv",
# #     BASE_DIR / "qwen4b_self_correct_toxic_prompt_benchmark.csv",
# #     BASE_DIR / "mistral_7b_instruct_toxic_prompt_benchmark.csv",
# #     BASE_DIR / "deepseek_r1_8b_toxic_prompt_benchmark.csv",
# #     BASE_DIR / "deepseek_r1_8b_debiased_toxic_prompt_benchmark.csv",
# # ]

# # # method 优先顺序（存在就显示）
# # METHOD_ORDER = ["static","prompt_safety","self_correction"]

# # def load_one(path: Path) -> pd.DataFrame:
# #     df = pd.read_csv(path)
# #     for c in ["prompt_id","segment_id"]:
# #         if c in df.columns:
# #             df[c] = pd.to_numeric(df[c], errors="coerce")
# #     # bias_score 如果存在就转数值
# #     if "bias_score" in df.columns:
# #         df["bias_score"] = pd.to_numeric(df["bias_score"], errors="coerce")
# #     return df

# # def show_record(r: dict):
# #     bs = r.get("bias_score", None)
# #     if isinstance(bs, float) and np.isnan(bs):
# #         bs = None
# #     print(f"    bias_score={bs}")
# #     txt = str(r.get("generated_text",""))
# #     print(f"    snippet: {txt[:SNIP].replace('\\n',' ')}" + ("..." if len(txt)>SNIP else ""))
# #     print("    FULL:")
# #     print(txt)

# # # ====== 3) 主对比打印 ======
# # for fp in baseline_files:
# #     if not fp.exists():
# #         print("[skip missing]", fp.name)
# #         continue

# #     df = load_one(fp)

# #     # 只取 toxic_prompt（保险：如果 split 列不存在就不筛）
# #     if "split" in df.columns:
# #         df = df[df["split"] == SPLIT]

# #     model_key = df["model_key"].iloc[0] if "model_key" in df.columns and len(df)>0 else fp.stem
# #     print("\n" + "#"*120)
# #     print(f"BASELINE FILE: {fp.name}")
# #     print(f"model_key={model_key}")

# #     # 方法有哪些
# #     methods_present = sorted(set(df["method"].dropna().unique())) if "method" in df.columns else ["unknown"]
# #     methods = [m for m in METHOD_ORDER if m in methods_present] + [m for m in methods_present if m not in METHOD_ORDER]

# #     for pid in prompt_ids:
# #         dpid = df[df["prompt_id"] == pid].copy()
# #         if dpid.empty:
# #             print(f"\n[prompt_id={pid}] NOT FOUND in this file")
# #             continue

# #         # prompt 文本
# #         prompt_txt = dpid["prompt"].iloc[0] if "prompt" in dpid.columns else ""
# #         print("\n" + "="*120)
# #         print(f"[prompt_id={pid}]")
# #         if prompt_txt:
# #             print("PROMPT:", prompt_txt[:500].replace("\n"," ") + ("..." if len(prompt_txt)>500 else ""))

# #         # 按 method → segment 打印
# #         for m in methods:
# #             dm = dpid[dpid["method"] == m] if "method" in dpid.columns else dpid
# #             if dm.empty:
# #                 continue
# #             print("\n" + "-"*110)
# #             print(f"  METHOD = {m}")
# #             for sid in SEG_IDS:
# #                 ds = dm[dm["segment_id"] == sid]
# #                 if ds.empty:
# #                     continue
# #                 r = ds.iloc[0].to_dict()
# #                 print(f"\n  [segment_id={sid}]")
# #                 show_record(r)

# # print("\nDONE.")
# import numpy as np
# import pandas as pd
# from pathlib import Path
# import torch
# from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

# ROOT = Path("/content/gdrive/MyDrive/narrative_cl_exp2/exp_runs")
# BASE_DIR = ROOT / "baseline_outputs"

# # ====== 1) 填入你要对比的 prompt_id（来自“掉得最多”的那些）======
# prompt_ids = [117,65,3 ]   # <-- 例如 [12, 87, 203, 6, 45]
# assert len(prompt_ids) > 0, "请先把 prompt_ids 填进去"

# SPLIT = "toxic_prompt"
# SEG_IDS = [0,1,2,3]
# SNIP = 220

# # ====== 2) 要对比的 baseline 文件（存在就读，不存在就跳过）======
# baseline_files = [
#     BASE_DIR / "qwen3_4b_toxic_prompt_benchmark.csv",
#     BASE_DIR / "qwen4b_self_correct_toxic_prompt_benchmark.csv",
#     BASE_DIR / "mistral_7b_instruct_toxic_prompt_benchmark.csv",
#     BASE_DIR / "deepseek_r1_8b_toxic_prompt_benchmark.csv",
#     BASE_DIR / "deepseek_r1_8b_debiased_toxic_prompt_benchmark.csv",
# ]

# METHOD_ORDER = ["static","prompt_safety","self_correction"]

# # ============================================================
# # 3) 评分器：committee bias_score（tox_mean）
# # ============================================================
# # snlp (fast)
# snlp_tok = AutoTokenizer.from_pretrained("s-nlp/roberta_toxicity_classifier")
# snlp_mdl = AutoModelForSequenceClassification.from_pretrained("s-nlp/roberta_toxicity_classifier").to("cpu")
# snlp_mdl.eval()

# snlp_toxic_index = 1
# try:
#     id2label = getattr(snlp_mdl.config, "id2label", None) or {}
#     for k,v in id2label.items():
#         if isinstance(v,str) and "toxic" in v.lower():
#             snlp_toxic_index = int(k); break
# except Exception:
#     snlp_toxic_index = 1

# @torch.no_grad()
# def p_snlp(text: str) -> float:
#     enc = snlp_tok(str(text)[:2000], return_tensors="pt", truncation=True)
#     out = snlp_mdl(**enc)
#     probs = torch.softmax(out.logits, dim=-1)[0]
#     return float(probs[min(snlp_toxic_index, probs.shape[-1]-1)].item())

# # toxic-bert (pipeline)
# toxic_bert = None
# try:
#     toxic_bert = pipeline("text-classification", model="unitary/toxic-bert", device=-1, top_k=None)
# except Exception:
#     toxic_bert = None

# def p_toxicbert(text: str):
#     if toxic_bert is None:
#         return None
#     out = toxic_bert(str(text)[:2000], truncation=True)
#     if isinstance(out, list) and out:
#         lab = {d["label"].lower(): float(d["score"]) for d in out if "label" in d and "score" in d}
#         if "toxic" in lab:
#             return float(lab["toxic"])
#         return float(max(lab.values())) if lab else None
#     return None

# # unbiased-toxic-roberta (pipeline)
# unbiased = None
# try:
#     unbiased = pipeline("text-classification", model="unitary/unbiased-toxic-roberta", device=-1, top_k=None)
# except Exception:
#     unbiased = None

# def p_unbiased(text: str):
#     if unbiased is None:
#         return None
#     out = unbiased(str(text)[:2000], truncation=True)
#     if isinstance(out, list) and out:
#         lab = {d["label"].lower(): float(d["score"]) for d in out if "label" in d and "score" in d}
#         if "toxic" in lab:
#             return float(lab["toxic"])
#         return float(max(lab.values())) if lab else None
#     return None

# def score_committee(text: str):
#     a = p_snlp(text)
#     b = p_toxicbert(text)   # may be None
#     c = p_unbiased(text)    # may be None
#     vals = [v for v in [a,b,c] if v is not None]
#     if not vals:
#         vals = [a]
#     return {
#         "tox_a": float(a),
#         "tox_b": float(b) if b is not None else None,
#         "tox_c": float(c) if c is not None else None,
#         "tox_mean": float(sum(vals)/len(vals)),
#         "tox_max": float(max(vals)),
#         "tox_min": float(min(vals)),
#         "tox_n": int(len(vals)),
#     }

# # ============================================================
# # 4) (可选) trigger scorer：如果你已经定义了 bias_scorer，就打印它的 bias_score_trigger
# # ============================================================
# HAS_TRIGGER = "bias_scorer" in globals()

# def safe_float(x):
#     try:
#         if x is None: return None
#         x = float(x)
#         if np.isnan(x): return None
#         return x
#     except Exception:
#         return None

# def show_record_with_scoring(label, r: dict):
#     txt = str(r.get("generated_text",""))
#     # existing bias_score from CSV (if any)
#     bs_exist = safe_float(r.get("bias_score"))

#     # committee score (always)
#     tp = score_committee(txt)
#     bs_committee = tp["tox_mean"]

#     # trigger score (optional)
#     trig = None
#     if HAS_TRIGGER:
#         s, _, _ = bias_scorer.score(txt, eps=0.3)
#         trig = float(s["bias_score_trigger"])

#     print(f"    bias_score(existing) = {bs_exist}")
#     print(f"    bias_score(committee tox_mean) = {bs_committee:.4f}  | tox_a={tp['tox_a']:.4f} tox_b={tp['tox_b']} tox_c={tp['tox_c']}")
#     if trig is not None:
#         print(f"    bias_score_trigger(TTA trigger) = {trig:.4f}")

#     print(f"    snippet: {txt[:SNIP].replace('\\n',' ')}" + ("..." if len(txt)>SNIP else ""))
#     print("    FULL:")
#     print(txt)

# def load_one(path: Path) -> pd.DataFrame:
#     df = pd.read_csv(path)
#     for c in ["prompt_id","segment_id"]:
#         if c in df.columns:
#             df[c] = pd.to_numeric(df[c], errors="coerce")
#     if "bias_score" in df.columns:
#         df["bias_score"] = pd.to_numeric(df["bias_score"], errors="coerce")
#     return df

# # ============================================================
# # 5) 主对比打印（带评分）
# # ============================================================
# for fp in baseline_files:
#     if not fp.exists():
#         print("[skip missing]", fp.name)
#         continue

#     df = load_one(fp)
#     if "split" in df.columns:
#         df = df[df["split"] == SPLIT]

#     model_key = df["model_key"].iloc[0] if "model_key" in df.columns and len(df)>0 else fp.stem
#     print("\n" + "#"*120)
#     print(f"BASELINE FILE: {fp.name}")
#     print(f"model_key={model_key}")

#     methods_present = sorted(set(df["method"].dropna().unique())) if "method" in df.columns else ["unknown"]
#     methods = [m for m in METHOD_ORDER if m in methods_present] + [m for m in methods_present if m not in METHOD_ORDER]

#     for pid in prompt_ids:
#         dpid = df[df["prompt_id"] == pid].copy()
#         if dpid.empty:
#             print(f"\n[prompt_id={pid}] NOT FOUND in this file")
#             continue

#         prompt_txt = dpid["prompt"].iloc[0] if "prompt" in dpid.columns else ""
#         print("\n" + "="*120)
#         print(f"[prompt_id={pid}]")
#         if prompt_txt:
#             print("PROMPT:", prompt_txt[:500].replace("\n"," ") + ("..." if len(prompt_txt)>500 else ""))

#         for m in methods:
#             dm = dpid[dpid["method"] == m] if "method" in dpid.columns else dpid
#             if dm.empty:
#                 continue
#             print("\n" + "-"*110)
#             print(f"  METHOD = {m}")
#             for sid in SEG_IDS:
#                 ds = dm[dm["segment_id"] == sid]
#                 if ds.empty:
#                     continue
#                 r = ds.iloc[0].to_dict()
#                 print(f"\n  [segment_id={sid}]")
#                 show_record_with_scoring(f"{model_key}/{m}", r)

# print("\nDONE.")

In [ ]:
from pathlib import Path
import pandas as pd

# 你的输出目录（按你现有程序）
OUT_DIR = TTA_DIR  # exp_runs/tta_outputs

def run_exists(run_id: str, out_dir: Path) -> bool:
    csv_path  = out_dir / f"{run_id}.csv"
    jsonl_path = out_dir / f"{run_id}.jsonl"
    upd_path  = out_dir / f"{run_id}_updates.jsonl"
    meta_path = out_dir / f"{run_id}_meta.jsonl"

    # 必须都存在
    if not (csv_path.exists() and jsonl_path.exists() and upd_path.exists() and meta_path.exists()):
        return False

    # 必须非空（csv 至少1行，jsonl 至少1行）
    try:
        df = pd.read_csv(csv_path)
        if len(df) == 0:
            return False
    except Exception:
        return False

    if csv_path.stat().st_size < 200 or jsonl_path.stat().st_size < 200:
        return False

    return True

def run_or_skip(cfg: dict, out_dir: Path):
    run_id = cfg["run_id"]
    if run_exists(run_id, out_dir):
        log(f"[SKIP] {run_id} already exists.")
        return (out_dir / f"{run_id}.csv", True)

    log(f"[RUN ] {run_id}")
    out_csv, out_jsonl, upd_jsonl, meta_jsonl = run_main_experiments(**cfg)
    log(f"[DONE] {run_id} -> {out_csv.name}")
    return (out_csv, False)

# -----------------------
# Build ablation configs
# -----------------------
BASE = dict(
    prompts=toxic_prompts,
    split="toxic_prompt",
    n_segments=4,
    seg_tokens=128,
    epsilon=0.3,
    typed_safebank=True,
    multi_trigger=True,
    safe_k=2,

    update_kind="precond",      # will override per run
    precond_steps=10,
    lambda_reg=1e-3,
    precond_max=500.0,

    lr_sgd=5e-4,
    lr_adamw=3e-4,
    lr_precond=3e-4,

    clip_coef=1.0,
    max_delta_norm=0.5,
    max_len_update=256,
    max_grad_norm=1.0,

    flush_every=2,
    ppl_probe_n=16,
)

def mk_run_id(prefix: str, **kw):
    parts = [prefix] + [f"{k}{v}".replace(".", "p") for k,v in kw.items()]
    return "_".join(parts)

RUNS = []

# 1) method compare
RUNS += [
    {**BASE, "run_id": mk_run_id("qwen3", kind="precond", eps=0.3), "update_kind":"precond", "epsilon":0.3},
        # {**BASE, "run_id": mk_run_id("qwen3", kind="precond", eps=1.0), "update_kind":"precond", "epsilon":1.0},

    # {**BASE, "run_id": mk_run_id("qwen3", kind="sgd", eps=0.3),     "update_kind":"sgd",     "epsilon":0.3},
    # {**BASE, "run_id": mk_run_id("qwen3", kind="adamw", eps=0.3),   "update_kind":"adamw",   "epsilon":0.3},
    # {**BASE, "run_id": mk_run_id("qwen3", kind="always", eps=0.0),  "update_kind":"always_precond", "epsilon":0.0},
    {
        **BASE,
        "run_id": mk_run_id("qwen3", kind="sgd10", eps=0.3),
        "update_kind": "sgd_10",
        "epsilon": 0.3,
        "precond_steps": 10,   # 关键：用同一个 step 计数
    },

    # 10-step AdamW
    {
        **BASE,
        "run_id": mk_run_id("qwen3", kind="adamw10", eps=0.3),
        "update_kind": "adamw_10",
        "epsilon": 0.3,
        "precond_steps": 10,
    },
    ]

# 2) epsilon sweep (precond)
for eps in [0, 0.2, 0.25, 0.3]:
    RUNS.append({**BASE, "run_id": mk_run_id("qwen3", kind="precond", eps=eps), "update_kind":"precond", "epsilon":eps})

# 3) typed vs generic
for typed in [True, False]:
    RUNS.append({**BASE, "run_id": mk_run_id("qwen3", typed=int(typed)), "typed_safebank":typed, "update_kind":"precond"})

# 4) multi-trigger vs single-trigger
for multi in [True, False]:
    RUNS.append({**BASE, "run_id": mk_run_id("qwen3", multi=int(multi)), "multi_trigger":multi, "update_kind":"precond"})

# 5) length ablation
for seg_tokens in [64, 128, 256]:
    RUNS.append({**BASE, "run_id": mk_run_id("qwen3", tok=seg_tokens), "seg_tokens":seg_tokens, "update_kind":"precond"})

for nseg in [2, 4, 8]:
    RUNS.append({**BASE, "run_id": mk_run_id("qwen3", nseg=nseg), "n_segments":nseg, "update_kind":"precond"})

# RUNS.append({
#     **BASE,
#     "run_id": mk_run_id("qwen3", kind="precond", eps=1.0, nseg=8),
#     "update_kind": "precond",
#     "epsilon": 1.0,
#     "n_segments": 8,
# })
# -----------------------
# Execute with skip
# -----------------------
written = []
skipped = 0
for cfg in RUNS:
    out_csv, was_skipped = run_or_skip(cfg, OUT_DIR)
    if was_skipped:
        skipped += 1
    else:
        written.append(str(out_csv))

log(f"ALL DONE. skipped={skipped}, newly_run={len(written)}")


[2025-12-29 21:55:27] [SKIP] qwen3_kindprecond_eps0p3 already exists.
[2025-12-29 21:55:28] [SKIP] qwen3_kindsgd10_eps0p3 already exists.
[2025-12-29 21:55:29] [SKIP] qwen3_kindadamw10_eps0p3 already exists.
[2025-12-29 21:55:30] [SKIP] qwen3_kindprecond_eps0 already exists.
[2025-12-29 21:55:31] [SKIP] qwen3_kindprecond_eps0p2 already exists.
[2025-12-29 21:55:32] [SKIP] qwen3_kindprecond_eps0p25 already exists.
[2025-12-29 21:55:32] [SKIP] qwen3_kindprecond_eps0p3 already exists.
[2025-12-29 21:55:34] [SKIP] qwen3_typed1 already exists.
[2025-12-29 21:55:35] [SKIP] qwen3_typed0 already exists.
[2025-12-29 21:55:36] [SKIP] qwen3_multi1 already exists.
[2025-12-29 21:55:37] [SKIP] qwen3_multi0 already exists.
[2025-12-29 21:55:38] [SKIP] qwen3_tok64 already exists.
[2025-12-29 21:55:39] [SKIP] qwen3_tok128 already exists.
[2025-12-29 21:55:39] [SKIP] qwen3_tok256 already exists.
[2025-12-29 21:55:40] [SKIP] qwen3_nseg2 already exists.
[2025-12-29 21:55:41] [SKIP] qwen3_nseg4 already ex

In [ ]:
# # ========= DeepSeek 接入（复用你现有 run_main_experiments / bias_scorer / precond / 4段逻辑） =========
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM

# # 1) 定义 DeepSeek loader（风格与你的 load_qwen3 一致）
# DEEPSEEK_ID = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"  # 你环境里能用的HF id就填这个

# def load_deepseek():
#     tok = AutoTokenizer.from_pretrained(DEEPSEEK_ID, use_fast=True, trust_remote_code=True)
#     if tok.pad_token is None:
#         tok.pad_token = tok.eos_token

#     model = AutoModelForCausalLM.from_pretrained(
#         DEEPSEEK_ID,
#         torch_dtype=DTYPE,  # 复用你前面定义的 DTYPE
#         device_map="auto" if torch.cuda.is_available() else None,
#         trust_remote_code=True,
#     )
#     print(f"[DeepSeek] loaded: {DEEPSEEK_ID}")
#     return tok, model

# # 2) 临时把 load_qwen3 指到 load_deepseek（不改 run_main_experiments 内部任何逻辑）
# _load_qwen3_backup = load_qwen3
# load_qwen3 = load_deepseek

# try:
#     RUN_ID = "deepseek_tta_precond_toxic_eps0.3_seg4"

#     out_csv, out_jsonl, upd_jsonl, meta_jsonl = run_main_experiments(
#         run_id=RUN_ID,
#         prompts=toxic_prompts,          # 复用你已读取好的 toxic_prompts
#         split="toxic_prompt",
#         n_segments=4,                   # ✅ 4段
#         seg_tokens=128,                 # ✅ 每段128 tokens
#         epsilon=0.3,
#         typed_safebank=True,
#         multi_trigger=True,
#         safe_k=2,

#         update_kind="precond",          # ✅ 用 preconditioned update
#         precond_steps=10,
#         lambda_reg=1e-3,
#         precond_max=500.0,
#         lr_precond=3e-4,
#         clip_coef=1.0,
#         max_delta_norm=0.5,

#         max_len_update=256,
#         max_grad_norm=1.0,
#         flush_every=2,
#         ppl_probe_n=16,
#     )

#     print("Saved:", out_csv, out_jsonl, upd_jsonl, meta_jsonl)

# finally:
#     # 3) 恢复原来的 load_qwen3，避免影响你后续其它实验
#     load_qwen3 = _load_qwen3_backup


In [ ]:
# BASE_SAFE = dict(
#     prompts=toxic_prompts,
#     split="toxic_prompt",

#     # segments
#     n_segments=4,
#     seg_tokens=128,

#     # trigger: 更保守一些（减少触发率/触发次数）
#     epsilon=0.25,          # 从0.3降到0.25（更少触发/更少更新）
#     typed_safebank=True,
#     multi_trigger=False,   # 关键：避免一个prompt里连续多次更新
#     safe_k=2,

#     # update
#     update_kind="precond",
#     precond_steps=3,       # 从10降到3（强烈建议）
#     lambda_reg=1e-2,       # 从1e-3加到1e-2（更强约束LoRA）
#     precond_max=200.0,     # 从500降到200（避免预条件放大）

#     # learning rates
#     lr_sgd=5e-4,
#     lr_adamw=3e-4,
#     lr_precond=1e-4,       # 从3e-4降到1e-4（更稳）

#     # trust region / clipping
#     clip_coef=1.0,
#     max_delta_norm=0.2,    # 从0.5降到0.2（强约束单次更新幅度）
#     max_len_update=256,
#     max_grad_norm=1.0,

#     flush_every=2,
#     ppl_probe_n=16,
# )
# RUNS += [
#     # 1) 推荐主配方
#     {**BASE_SAFE, "run_id": mk_run_id("qwen3", kind="precond_safe", eps=0.25, st=3, lr=1e-4),
#      "update_kind":"precond"},

#     # 2) 稍微激进一点：eps=0.3（保持你原触发阈值，但依旧稳更新）
#     {**BASE_SAFE, "run_id": mk_run_id("qwen3", kind="precond_safe", eps=0.3, st=3, lr=1e-4),
#      "epsilon":0.3, "update_kind":"precond"},

#     # # 3) 允许多触发但仍然小步（测试“是不是 multi_trigger 在害你”）
#     # {**BASE_SAFE, "run_id": mk_run_id("qwen3", kind="precond_safe", eps=0.25, st=3, lr=1e-4, multi=1),
#     #  "multi_trigger":True, "update_kind":"precond"},

#     # 4) 更强正则/更小更新幅度（如果你仍看到触发后变差就用这个）
#     {**BASE_SAFE, "run_id": mk_run_id("qwen3", kind="precond_safer", eps=0.25, st=3, lr=1e-4, lam=5e-2, dn=0.1),
#      "lambda_reg":5e-2, "max_delta_norm":0.1, "update_kind":"precond"},
# ]
# written = []
# skipped = 0
# for cfg in RUNS:
#     out_csv, was_skipped = run_or_skip(cfg, OUT_DIR)
#     if was_skipped:
#         skipped += 1
#     else:
#         written.append(str(out_csv))

# log(f"ALL DONE. skipped={skipped}, newly_run={len(written)}")

In [ ]:
# BASE_AGGRESSIVE = dict(
#     prompts=toxic_prompts,
#     split="toxic_prompt",

#     # segments
#     n_segments=4,
#     seg_tokens=128,

#     # trigger（更容易触发）
#     epsilon=0.35,            # ↑ 比0.3还激进
#     typed_safebank=True,
#     multi_trigger=True,      # ↑ 放开多次更新
#     safe_k=1,                # ↓ 更少安全约束

#     # update
#     update_kind="precond",
#     precond_steps=12,        # ↑↑
#     lambda_reg=5e-4,         # ↓↓
#     precond_max=500.0,       # ↑ 放大预条件

#     # learning rates
#     lr_precond=4e-4,         # ↑
#     lr_sgd=5e-4,
#     lr_adamw=3e-4,

#     # trust region（放松）
#     clip_coef=1.0,
#     max_delta_norm=0.8,      # ↑↑
#     max_len_update=256,
#     max_grad_norm=1.0,

#     flush_every=2,
#     ppl_probe_n=16,
# )

# # BASE_VERY_AGGRESSIVE = dict(
# #     **BASE_AGGRESSIVE,

# #     epsilon=0.4,             # ↑ 极易触发
# #     precond_steps=16,        # ↑↑
# #     lr_precond=6e-4,         # ↑↑
# #     lambda_reg=1e-4,         # ↓↓↓
# #     max_delta_norm=1.0,      # ↑↑
# # )


# RUNS += [
#     # 激进版（主力）
#     {
#         **BASE_AGGRESSIVE,
#         "run_id": mk_run_id("qwen3", kind="precond_aggr", eps=0.35, st=12, lr=4e-4),
#         "update_kind": "precond",
#     },

#     # 极限验证版
#     # {
#     #     **BASE_VERY_AGGRESSIVE,
#     #     "run_id": mk_run_id("qwen3", kind="precond_very_aggr", eps=0.4, st=16, lr=6e-4),
#     #     "update_kind": "precond",
#     # },
# ]
# written = []
# skipped = 0
# for cfg in RUNS:
#     out_csv, was_skipped = run_or_skip(cfg, OUT_DIR)
#     if was_skipped:
#         skipped += 1
#     else:
#         written.append(str(out_csv))

# log(f"ALL DONE. skipped={skipped}, newly_run={len(written)}")

In [ ]:
BASE_PRECOND_THEORETICAL = dict(
    prompts=toxic_prompts,
    split="toxic_prompt",

    # segments
    n_segments=4,
    seg_tokens=128,

    # trigger：回到“可信触发区间”
    epsilon=0.18,            # ↓ 比0.25更严格，避免噪声触发
    typed_safebank=True,
    multi_trigger=False,     # ❗ 关键：避免累积 precond 误差
    safe_k=2,

    # update：少步、但“方向可信”
    update_kind="precond",
    precond_steps=4,         # ↓ 保证 curvature 近似仍然成立
    lambda_reg=1e-3,         # ↑ 稍微收紧，防止方向发散
    precond_max=150.0,       # ↓ 防止矩阵放大噪声

    # learning rates：刻意偏小
    lr_precond=2e-4,         # ↓ 比你 aggressive 小，但不是 safe 那么小
    lr_sgd=5e-4,
    lr_adamw=3e-4,

    # trust region：严格
    clip_coef=1.0,
    max_delta_norm=0.25,     # ❗ 非常重要：保证在 local region
    max_len_update=256,
    max_grad_norm=1.0,

    flush_every=2,
    ppl_probe_n=16,
)
RUNS += [
    {
        **BASE_PRECOND_THEORETICAL,
        "run_id": mk_run_id("qwen3", kind="precond_theory", eps=0.18, st=4, lr=2e-4),
        "update_kind": "precond",
    },
]
written = []
skipped = 0
for cfg in RUNS:
    out_csv, was_skipped = run_or_skip(cfg, OUT_DIR)
    if was_skipped:
        skipped += 1
    else:
        written.append(str(out_csv))

log(f"ALL DONE. skipped={skipped}, newly_run={len(written)}")

[2025-12-30 02:47:00] [SKIP] qwen3_kindprecond_eps0p3 already exists.
[2025-12-30 02:47:00] [SKIP] qwen3_kindsgd10_eps0p3 already exists.
[2025-12-30 02:47:00] [SKIP] qwen3_kindadamw10_eps0p3 already exists.
[2025-12-30 02:47:00] [SKIP] qwen3_kindprecond_eps0 already exists.
[2025-12-30 02:47:00] [SKIP] qwen3_kindprecond_eps0p2 already exists.
[2025-12-30 02:47:00] [SKIP] qwen3_kindprecond_eps0p25 already exists.
[2025-12-30 02:47:00] [SKIP] qwen3_kindprecond_eps0p3 already exists.
[2025-12-30 02:47:00] [SKIP] qwen3_typed1 already exists.
[2025-12-30 02:47:00] [SKIP] qwen3_typed0 already exists.
[2025-12-30 02:47:00] [SKIP] qwen3_multi1 already exists.
[2025-12-30 02:47:00] [SKIP] qwen3_multi0 already exists.
[2025-12-30 02:47:00] [SKIP] qwen3_tok64 already exists.
[2025-12-30 02:47:00] [SKIP] qwen3_tok128 already exists.
[2025-12-30 02:47:00] [SKIP] qwen3_tok256 already exists.
[2025-12-30 02:47:00] [SKIP] qwen3_nseg2 already exists.
[2025-12-30 02:47:00] [SKIP] qwen3_nseg4 already ex

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

[2025-12-30 02:47:05] [Qwen3] loaded device=cuda:0 dtype=torch.bfloat16 offload=False
trainable params: 11,796,480 || all params: 4,034,264,576 || trainable%: 0.2924
[2025-12-30 02:48:31] [Main Flush] qwen3_kindprecond_theory_eps0p18_st4_lr0p0002 prompts 2/300
[2025-12-30 02:49:56] [Main Flush] qwen3_kindprecond_theory_eps0p18_st4_lr0p0002 prompts 4/300
[2025-12-30 02:51:21] [Main Flush] qwen3_kindprecond_theory_eps0p18_st4_lr0p0002 prompts 6/300
[2025-12-30 02:52:45] [Main Flush] qwen3_kindprecond_theory_eps0p18_st4_lr0p0002 prompts 8/300
[2025-12-30 02:54:09] [Main Flush] qwen3_kindprecond_theory_eps0p18_st4_lr0p0002 prompts 10/300
[2025-12-30 02:55:34] [Main Flush] qwen3_kindprecond_theory_eps0p18_st4_lr0p0002 prompts 12/300
[2025-12-30 02:56:59] [Main Flush] qwen3_kindprecond_theory_eps0p18_st4_lr0p0002 prompts 14/300
[2025-12-30 02:58:23] [Main Flush] qwen3_kindprecond_theory_eps0p18_st4_lr0p0002 prompts 16/300
[2025-12-30 02:59:47] [Main Flush] qwen3_kindprecond_theory_eps0p18_st